In [6]:
%pip install langchain langchain-openai langchain_chroma langchain_community langchainhub

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/19.0 MB ? eta -:--:--
   ------ --------------------------------- 3.1/19.0 MB 15.2 MB/s eta 0:00:02
   --------------- ------------------------ 7.3/19.0 MB 17.7 MB/s eta 0:00:01
   ----------------------- ---------------- 11.3/19.0 MB 18.1 MB/s eta 0:00:01
   --------------------------------- ------ 15.7/19.0 MB 18.8 MB/s eta 0:


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()
 
os.environ["SECRET"] = getpass.getpass()
 
token = os.getenv("SECRET")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"
 
from langchain_openai import ChatOpenAI
 
llm = ChatOpenAI(model=model, base_url=endpoint, openai_api_key=token)

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2017-06-21-overview/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
